# Precipitation Variability Across Timescales

This notebook demonstrates how to use the precipitation variability metrics driver. 

This notebook should be run in an environment with python, jupyterlab, pcmdi metrics package, and cdat installed. It is expected that you have downloaded the sample data as demonstrated in [the download notebook](Demo_0_download_data.ipynb).  

The following cell reads in the choices you made during the download data step:

In [1]:
from user_choices import demo_data_directory, demo_output_directory

## Basic Use

Use the `--help` flag for assistance with the precip variability driver:

In [2]:
%%bash
variability_across_timescales_PS_driver.py --help

usage: variability_across_timescales_PS_driver.py [-h]
                                                  [--parameters PARAMETERS]
                                                  [--diags OTHER_PARAMETERS [OTHER_PARAMETERS ...]]
                                                  [--mip MIP] [--mod MOD]
                                                  [--var VAR] [--frq FRQ]
                                                  [--modpath MODPATH]
                                                  [--results_dir RESULTS_DIR]
                                                  [--case_id CASE_ID]
                                                  [--prd PRD [PRD ...]]
                                                  [--fac FAC]
                                                  [--nperseg NPERSEG]
                                                  [--noverlap NOVERLAP]
                                                  [--ref REF] [--cmec]
                                                  [--no_

Settings are specified in a separate parameter file, which is printed here:

In [3]:
# print parameter file
with open("basic_precip_variability_param.py") as f:
    print(f.read())

mip = "cmip5"
exp = "historical"
mod = "GISS-E2-H"
var = "pr"
frq = "day"
modpath = 'demo_data_standard/CMIP5_demo_precip_var/'
results_dir = 'demo_output/precip_variability/GISS-E2-H/'
prd = [2000,2001]  # analysis period
fac = 86400  # factor to make unit of [mm/day]

# length of segment in power spectra (~10 years)
# shortened to 2 years for demo purposes
nperseg = 2 * 365
# length of overlap between segments in power spectra (~5 years)
# shortened to 1 year for demo purposes
noverlap = 1 * 365

# flag for cmec formatted JSON
cmec = False



The parameter file is passed to the driver using the `-p` flag, similar to other PMP metrics. The next cell runs the driver as a subprocess:

In [4]:
%%bash
variability_across_timescales_PS_driver.py -p basic_precip_variability_param.py

demo_data_standard/CMIP5_demo_precip_var/
GISS-E2-H
[2000, 2001]
730 365
demo_output/precip_variability/GISS-E2-H/
demo_output/precip_variability/GISS-E2-H/
demo_output/precip_variability/GISS-E2-H/
# of data: 1
['GISS-E2-H.r6i1p1']
GISS-E2-H.r6i1p1 365_day
Complete regridding from (365, 90, 144) to (365, 90, 180)
2000 (365, 90, 180)
Complete regridding from (365, 90, 144) to (365, 90, 180)
2001 (730, 90, 180)
Complete calculating climatology and anomaly for calendar of 365_day
Complete power spectra (segment:  730  nps: 1.0 )
Complete domain and frequency average of spectral power
Complete power spectra (segment:  730  nps: 1.0 )
Complete domain and frequency average of spectral power


INFO::2021-07-09 12:25::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/demo_output/precip_variability/GISS-E2-H/PS_pr.day_regrid.180x90_area.freq.mean_GISS-E2-H.r6i1p1.json


Running the precipitation variability driver produces three output files, found in the demo output directory:  
Spatial pattern of spectral power (forced variability) (netCDF)   
Spatial pattern of spectral power (unforced variability) (netCDF)  
Precipitation variability metrics (forced and unforced) (JSON)  

In [5]:
!ls {demo_output_directory + "/precip_variability/GISS-E2-H"}

PS_pr.day_regrid.180x90_GISS-E2-H.r6i1p1.nc
PS_pr.day_regrid.180x90_GISS-E2-H.r6i1p1_unforced.nc
PS_pr.day_regrid.180x90_area.freq.mean_GISS-E2-H.r6i1p1.json


The next cell displays the metrics from the JSON file.

In [6]:
import json
import os
output_path = os.path.join(demo_output_directory,"precip_variability/GISS-E2-H/PS_pr.day_regrid.180x90_area.freq.mean_GISS-E2-H.r6i1p1.json")
with open(output_path) as f:
    metric = json.load(f)["RESULTS"]
print(json.dumps(metric, indent=2))

{
  "GISS-E2-H.r6i1p1": {
    "forced": {
      "Land_30N50N": {
        "annual": 1.1587082092242942,
        "semi-annual": 0.41744881229451547
      },
      "Land_30S30N": {
        "annual": 6.971767234863867,
        "semi-annual": 1.1463392061273523
      },
      "Land_50S30S": {
        "annual": 0.6081871639608728,
        "semi-annual": 0.30708232514267686
      },
      "Land_50S50N": {
        "annual": 4.864266970917859,
        "semi-annual": 0.880099077324579
      },
      "Ocean_30N50N": {
        "annual": 1.573720339534381,
        "semi-annual": 0.31178472575620936
      },
      "Ocean_30S30N": {
        "annual": 4.868478624745868,
        "semi-annual": 1.485758806678225
      },
      "Ocean_50S30S": {
        "annual": 0.548059204349184,
        "semi-annual": 0.18055034836612316
      },
      "Ocean_50S50N": {
        "annual": 3.513253986567687,
        "semi-annual": 1.0538758410901587
      },
      "Total_30N50N": {
        "annual": 1.3812039923006247,


## Command line parameters with Obs data

To calculate the precipitation variability ratio, we also need results for a reference dataset. This example shows how to use the command line to call the `variability_across_timescales_PS_driver` using daily obs precipitation data. The command line arguments will overwrite values that are in the parameter file.  

In this example, the `mod` variable is set to an obs file name so that only a single file is used as input.

In [7]:
%%bash
variability_across_timescales_PS_driver.py -p basic_precip_variability_param.py \
--mip 'obs' \
--mod 'pr.day.GPCP-IP.BE.gn.v20200719.1998-1999.xml' \
--modpath 'demo_data_standard/PCMDIobs2/atmos/day/pr/GPCP-IP/gn/v20200719/' \
--results_dir 'demo_output/precip_variability/GPCP-IP/' \
--prd 1998 1999

demo_data_standard/PCMDIobs2/atmos/day/pr/GPCP-IP/gn/v20200719/
pr.day.GPCP-IP.BE.gn.v20200719.1998-1999.xml
[1998, 1999]
730 365
demo_output/precip_variability/GPCP-IP/
demo_output/precip_variability/GPCP-IP/
demo_output/precip_variability/GPCP-IP/
# of data: 1
['GPCP-IP']
GPCP-IP gregorian
Complete regridding from (365, 180, 360) to (365, 90, 180)
1998 (365, 90, 180)
Complete regridding from (365, 180, 360) to (365, 90, 180)
1999 (730, 90, 180)
Complete calculating climatology and anomaly for calendar of gregorian
Complete power spectra (segment:  730  nps: 1.0 )
Complete domain and frequency average of spectral power
Complete power spectra (segment:  730  nps: 1.0 )
Complete domain and frequency average of spectral power


/Users/ordonez4/miniconda3/envs/test_pcmdi/lib/python3.8/site-packages/pcmdi_metrics-v1.2.1_756_gac3a8d11-py3.8.egg/pcmdi_metrics/precip_variability/lib/lib_variability_across_timescales.py:229: UserWarning: Warning: converting a masked element to nan.
INFO::2021-07-09 12:26::pcmdi_metrics:: Results saved to a json file: /Users/ordonez4/Documents/git/pcmdi_metrics/doc/jupyter/Demo/demo_output/precip_variability/GPCP-IP/PS_pr.day_regrid.180x90_area.freq.mean_GPCP-IP.json


## Precipitation Variability Ratio

Once model and observational results are available, the ratio of those results can be derived. A script called "calc_ratio.py" is provided in the precip_variability codebase. This script can be called with three arguments to generate the ratio.  
`ref`: path to obs results JSON  
`modpath`: directory containing model results JSONS (not CMEC formatted JSONs)  
`results_dir`: directory for calc_ratio.py results

In [8]:
%%bash
python ../../../pcmdi_metrics/precip_variability/scripts_pcmdi/calc_ratio.py \
--ref demo_output/precip_variability/GPCP-IP/PS_pr.day_regrid.180x90_area.freq.mean_GPCP-IP.json \
--modpath demo_output/precip_variability/GISS-E2-H/ \
--results_dir demo_output/precip_variability/ratio/

reference:  demo_output/precip_variability/GPCP-IP/PS_pr.day_regrid.180x90_area.freq.mean_GPCP-IP.json
modpath:  demo_output/precip_variability/GISS-E2-H/
outdir:  demo_output/precip_variability/ratio/
['demo_output/precip_variability/GISS-E2-H/PS_pr.day_regrid.180x90_area.freq.mean_GISS-E2-H.r6i1p1.json']
Complete  GISS-E2-H.r6i1p1
Complete all


This outputs one JSON file in the `results_dir` folder. The results in this file are shown below.

In [9]:
output_path = os.path.join(demo_output_directory,"precip_variability/ratio/PS_pr.day_regrid.180x90_area.freq.mean_GISS-E2-H.r6i1p1.json")
with open(output_path) as f:
    metric = json.load(f)["RESULTS"]
print(json.dumps(metric, indent=2))

{
  "GISS-E2-H.r6i1p1": {
    "forced": {
      "Land_30N50N": {
        "annual": 1.1537856241415303,
        "semi-annual": 1.7543960179382367
      },
      "Land_30S30N": {
        "annual": 1.3152720576800716,
        "semi-annual": 1.4256044893620052
      },
      "Land_50S30S": {
        "annual": 1.175809836833849,
        "semi-annual": 2.291683414592525
      },
      "Land_50S50N": {
        "annual": 1.1763458183200783,
        "semi-annual": 1.3574907470709596
      },
      "Ocean_30N50N": {
        "annual": 1.0003034577121472,
        "semi-annual": 0.6190454959638267
      },
      "Ocean_30S30N": {
        "annual": 1.6110406656682628,
        "semi-annual": 1.862277868902164
      },
      "Ocean_50S30S": {
        "annual": 1.3540300994055696,
        "semi-annual": 0.975470587624899
      },
      "Ocean_50S50N": {
        "annual": 1.5478170476412825,
        "semi-annual": 1.6797755428644254
      },
      "Total_30N50N": {
        "annual": 1.032701509496752,
 